In [1]:
import argparse
import os
import torch
import numpy as np
import random
import pandas as pd


from exp.exp_NRU_RBN import Exp_NRU_RBN
%matplotlib inline

parser = argparse.ArgumentParser(description='Diff_TS Forecasting')

parser.add_argument('--model', type=str, default='Diff_TS',help='model of experiment, options: [NRU_RBN]')
value="ETTh1" #ETTh1,ETTm2,electrans,weather,exchange
parser.add_argument('--data', type=str, default=value, help='data')
parser.add_argument('--root_path', type=str, default=os.path.abspath('../')+'/datasets', help='root path of the data file')
parser.add_argument('--data_path', type=str, default=value+'.csv', help='data file')    #electrans.csv
parser.add_argument('--features', type=str, default='M', help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h', help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default= 'model_parameters/'+value, help='location of model checkpoints')

parser.add_argument('--seq_len', type=int, default=int(24*7), help='input sequence length of Informer encoder')
parser.add_argument('--label_len', type=int, default=int(24*7) , help='start token length of Informer decoder')
parser.add_argument('--pred_len', type=int, default=24, help='prediction sequence length')

parser.add_argument('--enc_in', type=int, default=1, help='encoder input size')
parser.add_argument('--dec_in', type=int, default=1, help='decoder input size')
parser.add_argument('--c_out', type=int, default=1, help='output size')
parser.add_argument('--d_model', type=int, default=256, help='dimension of model')
parser.add_argument('--d_ff', type=int, default=256, help='dimension of fcn')
parser.add_argument('--padding', type=int, default=0, help='padding type')
parser.add_argument('--distil', action='store_false', help='whether to use distilling in encoder, using this argument means not using distilling', default=False)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF', help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu',help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder',default=False)
parser.add_argument('--do_predict', action='store_false', help='whether to predict unseen future data')
parser.add_argument('--mix', action='store_false', help='use mix attention in generative decoder', default=True)
parser.add_argument('--cols', type=str, nargs='+', help='certain cols from the data files as the input features')
parser.add_argument('--num_workers', type=int, default=8, help='data loader num workers')
parser.add_argument('--itr', type=int, default=1, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=20, help='train epochs')
parser.add_argument('--batch_size', type=int, default=32, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=5, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test',help='exp description')
parser.add_argument('--loss', type=str, default='mse',help='loss function')
parser.add_argument('--lradj', type=str, default='type1',help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)
parser.add_argument('--inverse', action='store_true', help='inverse output data', default=False)
parser.add_argument('--likeloss', action='store_true', help='likeloss', default=True)
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3',help='device ids of multile gpus')
parser.add_argument('--lossregularizer', type=bool, default=True, help='Use loss regularizer')
parser.add_argument('--n_times', type=int, default=50, help='diffusion steps')
parser.add_argument('--sampling', type=bool, default=False, help='sampling of outcomes')
parser.add_argument('--sampling_times', type=int, default=50, help='number of outcomes')

args = parser.parse_args("")
args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

if args.use_gpu and args.use_multi_gpu:
    args.devices = args.devices.replace(' ','')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

data_parser = {
    'exchange':{'data':'exchange_all.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1],'freqin':'d'},
    'ETTh1':{'data':'ETTh1.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1],'freqin':'h'},
    'ETTh2':{'data':'ETTh2.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1],'freqin':'h'},
    'ETTm1':{'data':'ETTm1.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1],'freqin':'h'},
    'ETTm2':{'data':'ETTm2.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1],'freqin':'t'},
    'weather':{'data':'weather.csv','T':'OT','M':[12,12,12],'S':[1,1,1],'MS':[12,12,1],'freqin':'h'},
    'electrans':{'data':'electrans.csv','T':'OT','M':[316,316,316],'S':[1,1,1],'MS':[316,316,1],'freqin':'h'},
}
if args.data in data_parser.keys():
    data_info = data_parser[args.data]
    args.data_path = data_info['data']
    args.target = data_info['T']
    args.enc_in, args.dec_in, args.c_out = data_info[args.features]
    args.freq=data_info['freqin']

args.detail_freq = args.freq
args.freq = args.freq[-1:]
print('Args in experiment:')
print(args)
torch.cuda.empty_cache()


Args in experiment:
Namespace(model='Diff_TS', data='ETTh1', root_path='/home/uservan/Desktop/TS_Diff/datasets', data_path='ETTh1.csv', features='M', target='OT', freq='h', checkpoints='model_parameters/ETTh1', seq_len=168, label_len=168, pred_len=24, enc_in=7, dec_in=7, c_out=7, d_model=256, d_ff=256, padding=0, distil=False, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=True, mix=True, cols=None, num_workers=8, itr=1, train_epochs=20, batch_size=32, patience=5, learning_rate=0.0001, des='test', loss='mse', lradj='type1', use_amp=False, inverse=False, likeloss=True, use_gpu=True, gpu=0, use_multi_gpu=False, devices='0,1,2,3', lossregularizer=True, n_times=50, sampling=False, sampling_times=50, detail_freq='h')


In [ ]:
#test&train
torch.cuda.empty_cache()

Exp = Exp_NRU_RBN
def runtrain(args):
        mape_list=[]
        mae_list=[]
        rmse_list=[]
        dtw_list=[]
        res_list=[]
        shape_list=[]
        corr_list=[]
        crps_list=[]
        mse_list=[]
        crps_varlist=[]
        print(args.label_len)
        print(args.pred_len)
        index =args.model +"_"+str(args.label_len)+"_"+str(args.pred_len)
        resultsMAE,resultsCRPS,resultsMSE=[],[],[]
        mape_list,mae_list,rmse_list=[],[],[]
        dtw_list,res_list,shape_list=[],[],[]
        corr_list,crps_list,mse_list,crps_varlist=[],[],[],[]
        for ii in range(args.itr):
               
                # setting record of experiments
                setting = '{}_d{}_ft{}_sl{}_ll{}_pl{}_dm{}_df{}_eb{}_{}_lossreg_{}_n_times_{}_ii_{}'.format(
                        index, args.data, args.features,args.seq_len, args.label_len, args.pred_len,
                        args.d_model, args.d_ff,
                        args.embed, 0,str(args.lossregularizer),args.n_times,args.reg)
                
                exp = Exp(args) # set experiments
                print('>>>>>>>start training later {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
                _,tepochloss,tdataloss,tregloss,tvalepochloss,tvaldataloss,tvalregloss=exp.train(setting)
                torch.cuda.empty_cache()
                resultout,crpsret,sqtrue,sqpred,sigmaouts=exp.test(setting,True)
                
                mae,mse,rmse,mspe,shape =resultout

                print(setting)
                mse_list.append(mse)
                mse_np=np.array(mse_list)
                mae_list.append(mae)
                mae_np=np.array(mae_list)
                crps_list.append(np.mean(crpsret))
                #crps_varlist.append(np.var(crpsret))
                crps_np=np.array(crps_list)
                #crps_varlist_np=np.array(crps_varlist)
        
                print("predlen:"+str(args.pred_len)+",CRPS_L:"+str(len(crps_np))+",AVE:"+str(np.round(np.mean(crps_np),3))+"±"+str(np.round(np.std(crps_np),4)))
                print("predlen:"+str(args.pred_len)+",MSE_L:"+str(len(mse_np))+",AVE:"+str(np.round(np.mean(mse_np),3))+"±"+str(np.round(np.std(mse_np),4)))
                print("predlen:"+str(args.pred_len)+",MAE_L:"+str(len(mae_np))+",AVE:"+str(np.round(np.mean(mae_np),3))+"±"+str(np.round(np.std(mae_np),4)))
                resultsCRPS.append(str(ii)+"_"+"predlen:"+str(args.pred_len)+",CRPS_L:"+str(len(crps_np))+",AVE:"+str(np.round(np.mean(crps_np),3))+"±"+str(np.round(np.std(crps_np),4)))
                resultsMSE.append(str(ii)+"_"+"predlen:"+str(args.pred_len)+",MSE_L:"+str(len(mse_np))+",AVE:"+str(np.round(np.mean(mse_np),3))+"±"+str(np.round(np.std(mse_np),4)))
                resultsMAE.append(str(ii)+"_"+"predlen:"+str(args.pred_len)+",MAE_L:"+str(len(mae_np))+",AVE:"+str(np.round(np.mean(mae_np),3))+"±"+str(np.round(np.std(mae_np),4)))
                f= open(os.path.abspath('')+"/results/"+str(args.data)+"/"+str(args.pred_len)+"_d_ff"+str(args.d_ff)+"_ntimes"+str(args.n_times)+"_"+str(args.data)+".txt","w+")
                f.write(str(setting)+"\n"+str(resultsCRPS) +"\n"+str(resultsMSE)+"\n"+str(resultsMAE))
                f.close()
                sqtrue=0
                sqpred=0
        torch.cuda.empty_cache()
        return sqtrue,sqpred

for nn in range(3,4):
        valueindex=['ETTh1','exchange','ETTm2','electrans']
        args.data =valueindex[nn]
        args.checkpoints='model_parameters/'+valueindex[nn]
        
        data_parser = {
        'exchange':{'data':'exchange_all.csv','T':'OT','M':[8,8,8],'S':[1,1,1],'MS':[8,8,1],'freqin':'d'},
        'ETTh1':{'data':'ETTh1.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1],'freqin':'h'},
        'ETTh2':{'data':'ETTh2.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1],'freqin':'h'},
        'ETTm1':{'data':'ETTm1.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1],'freqin':'h'},
        'ETTm2':{'data':'ETTm2.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1],'freqin':'t'},
        'weather':{'data':'weather.csv','T':'OT','M':[12,12,12],'S':[1,1,1],'MS':[12,12,1],'freqin':'h'},
        'electrans':{'data':'electrans.csv','T':'OT','M':[32,32,32],'S':[1,1,1],'MS':[321,321,1],'freqin':'h'},
        }
        if args.data in data_parser.keys():
                data_info = data_parser[args.data]
                args.data_path = data_info['data']
                args.target = data_info['T']
                args.enc_in, args.dec_in, args.c_out = data_info[args.features]
                args.freq=data_info['freqin']
        
        args.detail_freq = args.freq
        args.freq = args.freq[-1:]
        print('Args in experiment:')
        print(args)
        for i in range(1,3,1):
                torch.cuda.empty_cache()
                fix_seed = 1
                torch.manual_seed(fix_seed)
                np.random.seed(fix_seed)
                args.d_ff=256
                args.d_model=256
                args.reg=100
                items= [[168,24],[168,96],[336,168],[720,336]] #for ETTh1
                if i==2:
                        args.batch_size=16
                elif i>2:
                        args.batch_size=8
                else:
                        args.batch_size=16    
                args.n_times=100
                args.label_len =items[i][0]
                args.seq_len=args.label_len
                args.pred_len =items[i][1]
                print(args.label_len)
                print(args.pred_len)
                sqtrue,sqpred=runtrain(args)


Args in experiment:
Namespace(model='Diff_TS', data='electrans', root_path='/home/uservan/Desktop/TS_Diff/datasets', data_path='electrans.csv', features='M', target='OT', freq='h', checkpoints='model_parameters/electrans', seq_len=168, label_len=168, pred_len=24, enc_in=32, dec_in=32, c_out=32, d_model=256, d_ff=256, padding=0, distil=False, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=True, mix=True, cols=None, num_workers=8, itr=1, train_epochs=20, batch_size=32, patience=5, learning_rate=0.0001, des='test', loss='mse', lradj='type1', use_amp=False, inverse=False, likeloss=True, use_gpu=True, gpu=0, use_multi_gpu=False, devices='0,1,2,3', lossregularizer=True, n_times=50, sampling=False, sampling_times=50, detail_freq='h')
168
96
168
96
Use GPU: cuda:0
Trainable Parameters:3919457
Trainable Parameters:298016
>>>>>>>start training later Diff_TS_168_96_delectrans_ftM_sl168_ll168_pl96_dm256_df256_ebtimeF_0_lossreg_True_n_times_100_ii_100>>>>>>>>>>>>

  0%|          | 0/1135 [00:00<?, ?it/s]